# 1. package importation

This script use **pandas** as main functional package

In [8]:
%load_ext blackcellmagic
import pandas as pd

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


## 2.1 Admin & Demographicks


In [9]:
admin1 = pd.read_csv('../Data/GHA/ADM1/admin.csv')
admin1_code_and_names = admin1[['3166-2 code','Subdivision name']]

In [10]:
# read data and get adm1 level demographic data
gha_demographic_raw_data = pd.read_excel("../Data/GHA/demographic/ghana.xlsx", "Sheet2")
gha_demographic_adm1_raw_data = gha_demographic_raw_data.loc[
    gha_demographic_raw_data["ADM_LEVEL"] == 1
]
gha_demographic_adm1_raw_data = gha_demographic_adm1_raw_data.set_index(
    keys="ADM1_NAME"
)
# select the most recent 2020 data
gha_demographic_adm1_raw_data_2020 = gha_demographic_adm1_raw_data.loc[
    :, "B0004_2020":"B80PL_2020"
]

In [38]:
gha_demographic_adm1_2020_population_total = gha_demographic_adm1_raw_data_2020.sum(axis=1)
gha_demographic_adm1_2020_population_median = gha_demographic_adm1_2020_population_total / 2
# expand columns, each column = to cumsum the previous columns
gha_demographic_adm1_2020_population_expanding = gha_demographic_adm1_raw_data_2020.expanding(axis=1).sum()
# expanded columns - median, the first non negative column is where median
gha_demographic_adm1_2020_raw_subtract_median = gha_demographic_adm1_2020_population_expanding.subtract(gha_demographic_adm1_2020_population_median, axis=0)
# get ratio in groups
# check the answer here
# https://stackoverflow.com/questions/38467749/find-first-non-zero-value-in-each-row-of-pandas-dataframe/38468180
gha_demographic_adm1_2020_median_ratio = gha_demographic_adm1_2020_raw_subtract_median.where(gha_demographic_adm1_2020_raw_subtract_median>0) / gha_demographic_adm1_raw_data_2020
gha_demographic_adm1_2020_median_ratio = gha_demographic_adm1_2020_median_ratio.bfill(1).iloc[:,0] * 4
# get group labels
gha_demographic_adm1_2020_median_group_value = gha_demographic_adm1_raw_data_2020 - gha_demographic_adm1_2020_population_expanding
gha_demographic_adm1_median_group_labels = (gha_demographic_adm1_2020_raw_subtract_median > 0).idxmax(axis=1)
# get first two string as the start of age group
gha_demographic_adm1_median_group_labels = gha_demographic_adm1_median_group_labels.apply(lambda x: float(x[1:3]))
# add age group with ratio
gha_demographic_adm1_2020_median = gha_demographic_adm1_median_group_labels + gha_demographic_adm1_2020_median_ratio

/var/folders/jw/1fjpkkvx6wz8tmmfqfklf4b80000gn/T/ipykernel_80308/4287399573.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.bfill will be keyword-only
  gha_demographic_adm1_2020_median_ratio = gha_demographic_adm1_2020_median_ratio.bfill(1).iloc[:,0] * 4


# Create `pandas` dataframe from the retrieved index and columns

In [43]:
columns = [
    "eta",
    "rho",
    "a0",
    "sigma2",
    "max_age",
    "rA",
    "rT",
    "rD",
    "rU",
    "rP",
    "dE",
    "delayGam",
    "cD",
    "cT",
    "cU",
    "gamma1",
    "d1",
    "dID",
    "ID0",
    "kD",
    "uD",
    "aD",
    "fD0",
    "gammaD",
    "alphaA",
    "alphaU",
    "b0",
    "b1",
    "dB",
    "IB0",
    "kB",
    "uB",
    "phi0",
    "phi1",
    "dCA",
    "IC0",
    "kC",
    "uCA",
    "PM",
    "dCM",
    "delayMos",
    "tau1",
    "tau2",
    "mu0",
    "Q0",
    "chi",
    "bites_Bed",
    "bites_Indoors",
    "muEL",
    "muLL",
    "muPL",
    "dEL",
    "dLL",
    "dPL",
    "gammaL",
    "km",
    "cm",
    "betaL",
    "num_int",
    "itn_cov",
    "irs_cov",
    "ITN_IRS_on",
    "DY",
    "d_ITN0",
    "r_ITN0",
    "r_ITN1",
    "r_IRS0",
    "d_IRS0",
    "irs_half_life",
    "itn_half_life",
    "IRS_interval",
    "ITN_interval",
]

gha = pd.DataFrame(columns=columns, index=gha_demographic_adm1_2020_median.index)

## 1. eta 
Death rate for expoential population distribtuion, i.e. 1/Mean Population Age. Default = 0.0001305

In [45]:
gha['eta'] = 1/(gha_demographic_adm1_2020_median*365)

## 2. rho
Age-dependent biting parameter. Default = 0.85
#' @param eta Death rate for expoential population distribtuion, i.e. 1/Mean Population Age. Default = 0.0001305
#' @param rho Age-dependent biting parameter. Default = 0.85
#' @param a0 Age-dependent biting parameter. Default = 2920
#' @param sigma2 Variance of the log heterogeneity in biting rates. Default = 1.67
#' @param max_age Maximum age in days. Default = 100*365
#' @param rA Rate of leaving asymptomatic infection. Default = 0.00512821
#' @param rT Rate of leaving treatment. Default = 0.2
#' @param rD Rate of leaving clinical disease. Default = 0.2
#' @param rU Rate of recovering from subpatent infection. Default = 0.00906627
#' @param rP Rate of leaving prophylaxis. Default = 0.06666667
#' @param dE Latent period of human infection. Default = 12
#' @param delayGam Lag from parasites to infectious gametocytes. Default = 12.5
#' @param cD Untreated disease contribution to infectiousness. Default = 0.0676909
#' @param cT Treated disease contribution to infectiousness. Default =   0.322 * cD
#' @param cU Subpatent disease contribution to infectiousness. Default = 0.006203
#' @param gamma1 Parameter for infectiousness of state A. Default = 1.82425
#' @param d1 Minimum probability due to maximum immunity. Default = 0.160527
#' @param dID Inverse of decay rate. Default = 3650
#' @param ID0 Scale parameter. Default = 1.577533
#' @param kD Shape parameter. Default = 0.476614
#' @param uD Duration in which immunity is not boosted. Default = 9.44512
#' @param aD Scale parameter relating age to immunity. Default = 8001.99
#' @param fD0 Time-scale at which immunity changes with age. Default = 0.007055
#' @param gammaD Shape parameter relating age to immunity. Default = 4.8183
#' @param alphaA PCR detection probability parameters state A. Default = 0.757
#' @param alphaU PCR detection probability parameters state U. Default = 0.186
#' @param b0 Maximum probability due to no immunity. Default = 0.590076
#' @param b1 Maximum relative reduction due to immunity. Default = 0.5
#' @param dB Inverse of decay rate. Default = 3650
#' @param IB0 Scale parameter. Default = 43.8787
#' @param kB Shape parameter. Default = 2.15506
#' @param uB Duration in which immunity is not boosted. Default = 7.19919
#' @param phi0 Maximum probability due to no immunity. Default = 0.791666
#' @param phi1 Maximum relative reduction due to immunity. Default = 0.000737
#' @param dCA Inverse of decay rate. Default = 10950
#' @param IC0 Scale parameter. Default = 18.02366
#' @param kC Shape parameter. Default = 2.36949
#' @param uCA Duration in which immunity is not boosted. Default = 6.06349
#' @param PM New-born immunity relative to mother’s. Default = 0.774368
#' @param dCM Inverse of decay rate of maternal immunity. Default = 67.6952
#' @param delayMos Extrinsic incubation period. Default = 10
#' @param tau1 Duration of host seeking, assumed to be constant between species. Default = 0.69
#' @param tau2 Duration of mosquito resting after feed. Default = 2.31
#' @param mu0 Daily mortality of adult mosquitos. Default = 0.132
#' @param Q0 Anthrophagy probability. Default = 0.92
#' @param chi Endophily probability. Default = 0.86
#' @param bites_Bed Percentage of bites indoors and in bed. Default = 0.89
#' @param bites_Indoors Percentage of bites indoors . Default = 0.97
#' @param muEL Per capita daily mortality rate of early stage larvae (low density). Default = 0.0338
#' @param muLL Per capita daily mortality rate of late stage larvae (low density). Default = 0.0348
#' @param muPL Per capita daily mortality rate of pupae. Default = 0.249
#' @param dEL Development time of early stage larvae. Default = 6.64
#' @param dLL Development time of late stage larvae. Default = 3.72
#' @param dPL Development time of pupae. Default = 0.643
#' @param gammaL Relative effect of density dependence on late instars relative to early instars. Default = 13.25
#' @param km Seasonal carrying capacity. Default = 11
#' @param cm Seasonal birth rate. Default = 0.05
#' @param betaL Number of eggs laid per day per mosquito. Default = 21.2
#' @param num_int Number of intervention parameters.  Default = 4
#' @param itn_cov The proportion of people that use an ITN. Default = 0
#' @param irs_cov The proportion of people living in houses that have been sprayed. Default = 0
#' @param ITN_IRS_on Time of ITN and IRS to be activated. Default = -1, i.e. never.
#' @param DY Duration of year (days). Default = 365
#' @param d_ITN0 Probability of dying with an encounter with ITN (max). Default = 0.41
#' @param r_ITN0 Probability of repeating behaviour with ITN (max). Default = 0.56
#' @param r_ITN1 Probability of repeating behaviour with ITN (min). Default = 0.24
#' @param r_IRS0 Probability of repeating behaviour with IRS (min). Default = 0.6
#' @param d_IRS0 Probability of dying with an encounter with IRS (max). Default = 1
#' @param irs_half_life IRS half life. Default =   0.5 * DY
#' @param itn_half_life ITN half life. Default =   2.64 * DY
#' @param IRS_interval How long before IRS is repeated, i.e. when IRS decay = 1. Default =   1 * DY
#' @param ITN_interval How long before ITN is repeated, i.e. when IRS decay = 1.  Default =   3 * DY

In [48]:
gha["rho"] = 0.85
gha["a0"] = 2920
gha["sigma2"] = 1.67
gha["max_age"] = 100 * 365
gha["rA"] = 0.00512821
gha["rT"] = 0.2
gha["rD"] = 0.2
gha["rU"] = 0.00906627
gha["rP"] = 0.06666667
# TODO check if this data is availble from the data folder
gha["dE"] = 12
gha["delayGam"] = 12.5
gha["cD"] = 0.0676909
gha["cT"] = 0.322 * gha["cD"]
gha['cU'] = 0.006203
gha['gamma1'] = 1.82425
gha['d1'] = 0.160527
gha['dID'] = 3650
gha['ID0'] = 1.577533
gha['kD'] = 0.476614
gha['uD'] = 9.44512
gha['aD'] = 8001.99
gha['fD0'] = 0.007055
gha['gammaD'] = 4.8183
# TODO 
gha['']

3. a0

## 2.2. Interventions


In [ ]:
GHA_Intervention_data = pd.read_excel('../Data/GHA/Interventions/GHA_Intervention_data.xlsx','Data template ')
GHA_Intervention_data

In [ ]:
pd.merge(admin1_code_and_names, GHA_Intervention_data, left_on='Subdivision name', right_on='adm1')